In [10]:
import tensorflow as tf

x1 = tf.constant(5)
x2 = tf.constant(6)

result = tf.multiply(x1,x2)

sess = tf.Session()
print(sess.run(result))


    


30


In [11]:
import tensorflow as tf

'''
input > weight > hidden layer 1 (activation function)  > weights > hidden l 2 (activation function) > weights > 
output layer

compare output to intended output > cost function (cross entropy)
optimization function(optimizer) > minimize cost (AdamOptimizer....SGD, AdaGrad)

backpropogation

feed forward+backprop = epoch

'''


'\ninput > weight > hidden layer 1 (activation function)  > weights > hidden l 2 (activation function) > weights > \noutput layer\n\ncompare output to intended output > cost function (cross entropy)\noptimization function(optimizer) > minimize cost (AdamOptimizer....SGD, AdaGrad)\n\nbackpropogation\n\nfeed forward+backprop = epoch\n\n'

In [25]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

n_nodes_hl1 = 300
n_nodes_hl2 = 200
n_nodes_hl3 = 100

n_classes = 10
batch_size = 100

#height x width
x = tf.placeholder('float',[None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    

    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'biases':tf.Variable(tf.random_normal([n_classes]))}
    
        # (input data * weights) + biases
        
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(  tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    #      learning_rate_default = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #cycles feed forward + backprop
    hm_epochs = 20
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction,1 ), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        
train_neural_network(x)
    


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch 0 completed out of 20 loss: 585853.8907318115
Epoch 1 completed out of 20 loss: 126639.99866867065
Epoch 2 completed out of 20 loss: 75780.45319366455
Epoch 3 completed out of 20 loss: 51372.74943470955
Epoch 4 completed out of 20 loss: 36072.39020574093
Epoch 5 completed out of 20 loss: 25825.30467939377
Epoch 6 completed out of 20 loss: 18712.25850987315
Epoch 7 completed out of 20 loss: 13735.063741649441
Epoch 8 completed out of 20 loss: 10257.865893954175
Epoch 9 completed out of 20 loss: 7403.767608346028
Epoch 10 completed out of 20 loss: 5391.57618905096
Epoch 11 completed out of 20 loss: 4413.210320613318
Epoch 12 completed out of 20 loss: 3397.028035644183
Epoch 13 completed out of 20 loss: 2724.9573218645965
Epoch 14 completed out of 20 loss: 2611.7317064917406
Epoch 15 completed ou

In [39]:

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter

lemmatizer = WordNetLemmatizer()
hm_lines = 10000000

def create_lexicon(pos,neg):
    lexicon = []
    for fi in [pos,neg]:
        with open(fi, 'r') as f:
            contents = f.readlines()
            for l in contents[:hm_lines]:
                all_words = word_tokenize(l.lower())
                lexicon += list(all_words)
                
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    print(len(l2))
    return l2

def sample_handling(sample, lexicon, classification):
    featureset=[]
    
    with open(sample, 'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1
                features = list(features)
                featureset.append([features, classification])
                
    return featureset

def create_feature_sets_and_labels(pos,neg,test_size=0.1):
    lexicon = create_lexicon(pos,neg)
    features = []
    features += sample_handling('pos.txt', lexicon,[1,0])
    features += sample_handling('neg.txt', lexicon,[0,1])
    random.shuffle(features)
    
    features = np.array(features)
    testing_size = int(test_size*len(features))
    
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x, train_y, test_x, test_y

if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')
    with open('sentiment_set.pickle', 'wb') as f:
        pickle.dump([train_x,train_y,test_x,test_y], f)


[nltk_data] Downloading package punkt to /Users/frank/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/frank/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
423


In [3]:
import tensorflow as tf

from deep_learning_neural_networks import create_feature_sets_and_labels
import numpy as np

train_x,train_y,test_x,test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100

#height x width
x = tf.placeholder('float',[None, len(train_x[0])])
y = tf.placeholder('float')

def neural_network_model(data):
    

    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'biases':tf.Variable(tf.random_normal([n_classes]))}
    
        # (input data * weights) + biases
        
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(  tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    #      learning_rate_default = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #cycles feed forward + backprop
    hm_epochs = 20
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            i = 0
            while i < len(train_x):
                start = i 
                end = i + batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                epoch_loss += c
                
                i += batch_size
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction,1 ), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:test_x, y:test_y}))
        
train_neural_network(x)
    


423
Epoch 0 completed out of 20 loss: 730503.8408737183
Epoch 1 completed out of 20 loss: 111575.7743692398
Epoch 2 completed out of 20 loss: 62349.23623275757
Epoch 3 completed out of 20 loss: 50111.205814540386
Epoch 4 completed out of 20 loss: 43829.947853565216
Epoch 5 completed out of 20 loss: 39830.41049528122
Epoch 6 completed out of 20 loss: 34853.34167611599
Epoch 7 completed out of 20 loss: 31793.373434901237
Epoch 8 completed out of 20 loss: 28193.914291620255
Epoch 9 completed out of 20 loss: 24710.750375539064
Epoch 10 completed out of 20 loss: 22081.29998409748
Epoch 11 completed out of 20 loss: 19765.11409842968
Epoch 12 completed out of 20 loss: 17627.389706641436
Epoch 13 completed out of 20 loss: 15594.471472531557
Epoch 14 completed out of 20 loss: 14147.026262104511
Epoch 15 completed out of 20 loss: 12184.336075142026
Epoch 16 completed out of 20 loss: 10818.940199971199
Epoch 17 completed out of 20 loss: 9757.307924956083
Epoch 18 completed out of 20 loss: 8684.18

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import pandas as pd

lemmatizer = WordNetLemmatizer()

def init_process(fin, fout):
    outfile = open(fout, 'a')
    with open(fin, buffering = 200000, encoding='latin-1') as f:
        try:
            for line in f:
                line = line.replace('"','')
                initial_polarity = line.split(',')[0]
                if initial_polarity == '0':
                    initial_polarity = [1,0]
                elif initial_polarity == '4':
                    initial_polarity = [0,1]
                    
                tweet = line.split(',')[-1]
                outline = str(initial_polarity)+':::'+tweet
                outfile.write(outline)
                
        except Exception as e:
                print(str(e))
    outfile.close()
    
init_process('training.1600000.processed.noemoticon.csv', 'train_set.csv')
init_process('testdata.manual.2009.06.14.csv', 'test_set.csv')



def create_lexicon(fin):
    lexicon = []
    with open(fin, 'r', buffering = 100000, encoding='latin-1') as f:
        try:
            counter = 1
            content = ''
            for line in f:
                counter += 1
                if (counter/2500.0).is_integer():
                    tweet = line.split(':::')[1]
                    content += ' '+tweet
                    words = word_tokenize(content)
                    words = [lemmatizer.lemmatize(i) for i in words]
                    lexicon = list(set(lexicon + words))
                    print(counter, len(lexicon))
                    
        except Exception as e:
            print(str(e))
            
    with open('lexicon.pickle', 'wb') as f:
        pickle.dump(lexicon, f)
        
create_lexicon('train_set.csv')

2500 26
5000 32
7500 41
10000 63
12500 85
15000 89
17500 91
20000 108
22500 113
25000 120
27500 128
30000 141
32500 159
35000 173
37500 177
40000 185
42500 198
45000 212
47500 224
50000 240
52500 253
55000 266
57500 267
60000 269
62500 280
65000 284
67500 291
70000 299
72500 306
75000 321
77500 329
80000 336
82500 345
85000 356
87500 358
90000 361
92500 374
95000 388
97500 399
100000 402
102500 412
105000 418
107500 425
110000 429
112500 438
115000 448
117500 450
120000 453
122500 460
125000 465
127500 466
130000 468
132500 470
135000 480
137500 490
140000 496
142500 500
145000 507
147500 514
150000 528
152500 532
155000 536
157500 540
160000 548
162500 556
165000 560
167500 562
170000 566
172500 571
175000 580
177500 583
180000 588
182500 594
185000 597
187500 600
190000 607
192500 611
195000 621
197500 624
200000 628
202500 630
205000 632
207500 641
210000 644
212500 648
215000 651
217500 659
220000 674
222500 684
225000 689
227500 694
230000 698
232500 700
235000 706
237500 714
2400

1697500 2638
1700000 2638
1702500 2638
1705000 2638
1707500 2638
1710000 2638
1712500 2638
1715000 2638
1717500 2638
1720000 2638
1722500 2638
1725000 2638
1727500 2638
1730000 2638
1732500 2638
1735000 2638
1737500 2638
1740000 2638
1742500 2638
1745000 2638
1747500 2638
1750000 2638
1752500 2638
1755000 2638
1757500 2638
1760000 2638
1762500 2638
1765000 2638
1767500 2638
1770000 2638
1772500 2638
1775000 2638
1777500 2638
1780000 2638
1782500 2638
1785000 2638
1787500 2638
1790000 2638
1792500 2638
1795000 2638
1797500 2638
1800000 2638
1802500 2638
1805000 2638
1807500 2638
1810000 2638
1812500 2638
1815000 2638
1817500 2638
1820000 2638
1822500 2638
1825000 2638
1827500 2638
1830000 2638
1832500 2638
1835000 2638
1837500 2638
1840000 2638
1842500 2638
1845000 2638
1847500 2638
1850000 2638
1852500 2638
1855000 2638
1857500 2638
1860000 2638
1862500 2638
1865000 2638
1867500 2638
1870000 2638
1872500 2638
1875000 2638
1877500 2638
1880000 2638
1882500 2638
1885000 2638
1887500 2638

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import pandas as pd


def shuffle_data(fin):
    df = pd.read_csv(fin, error_bad_lines=False)
    df = df.iloc[np.random.permutation(len(df))]
    print(df.head())
    df.to_csv('train_set_shuffled.csv', index=False)
    

def create_test_data_pickle(fin):
    feature_sets = []
    labels = []
    counter = 0
    with open(fin, buffering=20000) as f:
        for line in f:
            try:
                features = list(eval(line.split(':::')[0]))
                label = list(eval(line.split(':::')[1]))
                
                feature_sets.append(features)
                labels.append(label)
                counter += 1
            except:
                pass
        print(counter)
        feature_sets = np.array(feature_sets)
        labels = np.array(labels)
        
create_test_data_pickle('test_set.csv')

0


In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

n_nodes_hl1 = 500
n_nodes_hl2 = 500

n_classes = 2 

batch_size = 32

total_batches = int(1600000/batch_size)
hm_epochs = 10

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([2638, n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

output_layer = {'f_fum':None,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_classes])),
                  'bias':tf.Variable(tf.random_normal([n_classes]))}

def neural_network_model(data):
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)
    output = tf.matmul(l2, output_layer['weight']) + output_layer['bias']
    return output

saver = tf.train.Saver()
tf_log = 'tf.log'

def train_neural_network(x):
    prediction = neural_network_model(x)
    print(prediction)
    print('|||||||')
    print(y)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer)
        
        try:
            epoch = int(open(tf_log, 'r').read().split('\n')[-2]) + 1
            print('STARTING:', epoch)
        except:
            epoch = 1
            
        while epoch <= hm_epochs:
            if epoch != 1:
                saver.restore(sess,"model.ckpt")
                epoch_loss = 1
                with open('lexicon.pickle', 'rb') as f:
                    lexicon = pickle.load(f)
                with open('train_set_shuffled.csv', buffering = 200000, encoding='latin-1') as f:
                    batch_x = []
                    batch_y = []
                    batches_run = 0
                    for line in f:
                        label = line.split(':::')[0]
                        tweet = line.split(':::')[1]
                        current_words = word_tokenize(tweet.lower())
                        current_words = [lemmatizer.lemmatize(i) for i in current_words]
                        
                        features = np.zeros(len(lexicon))
                        
                        for word in current_words:
                            if word.lower() in lexicon:
                                index_vlaue = lexicon.index(word.lower())
                                features[index_value] += 1
                                line_x = list(features)
                                line_y = eval(label)
                                batch_x.append(line_x)
                                batch_y.append(line_y)
                                if len(batch_x) >= batch_size:
                                    _, c = sess.run([optimizer, cost], feed_dict={x: np.array(batch_x),
                                                                                 y: np.array(batch_y)})
                                    epoch_loss += c
                                    batch_x = []
                                    batch_y = []
                                    batches_run += 1
                                    print('Batch run:',batches_run, '/', total_batches, '| Epoch:', epoch,'| Batch Loss:', c,)
                                    
                    saver.save(sess, "model.ckpt")
                    print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
                    with open(tf_log, 'a') as f:
                        f.write(str(epoch) + '\n')
                    epoch += 1
                    
train_neural_network(x)